In [5]:
import numpy as np
import pandas as pd
import scipy.io

In [6]:
data = scipy.io.loadmat('Statistical Learning & Neural Networks/Lab1-SLNN2020/synthetic.mat')

In [7]:
pd.DataFrame({'x': data['knnClassify2dTest'][:,0], 'y': data['knnClassify2dTest'][:,1], 'label': data['knnClassify2dTest'][:,2]})

,x,y,label
0,3.0766,3.9354,1.0
1,0.3230,3.7768,1.0
2,2.2275,3.1865,1.0
3,2.1847,3.1470,1.0
4,2.0982,2.6313,1.0
...,...,...,...
95,-4.8813,-0.2345,2.0
96,-1.3670,-0.2383,2.0
97,-0.2258,1.4781,2.0
98,-2.0594,-1.1995,2.0
